In [2]:
import cv2
import mediapipe as mp
import numpy as np

# mp setup
mp_draw = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# quick angle calc
def knee_angle(p1, p2, p3):
    if p1.visibility < 0.8 or p2.visibility < 0.8 or p3.visibility < 0.8:
        return -1
    a = np.array([p1.x - p2.x, p1.y - p2.y, p1.z - p2.z])
    b = np.array([p3.x - p2.x, p3.y - p2.y, p3.z - p2.z])
    ang = np.degrees(np.arccos(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))))
    return 360 - ang if ang > 180 else ang

# leg pos: 1 squat, 2 mid, 3 up
def leg_pos(angle):
    if angle < 0: return 0
    if angle < 105: return 1
    if angle < 150: return 2
    return 3

# cam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("no cam!")
    exit()

# vars
count = 0
last_state = 9

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ok, frame = cap.read()
        if not ok:
            print("cam error")
            break

        frame = cv2.resize(frame, (960, 540))
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb.flags.writeable = False
        results = pose.process(rgb)
        rgb.flags.writeable = True

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark

            mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            r_ang = knee_angle(lm[24], lm[26], lm[28])
            l_ang = knee_angle(lm[23], lm[25], lm[27])

            rs = leg_pos(r_ang)
            ls = leg_pos(l_ang)
            state = rs * ls

            if state in (1, 9) and state != last_state:
                last_state = state
                if state == 1:
                    count += 1
                    print(f"squat! total: {count}")
            elif state == 0:
                if rs == 0: print("right leg lost")
                if ls == 0: print("left leg lost")

        else:
            print("step in view")

        cv2.putText(frame, f"Reps: {count}", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Squat Counter", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost

In [3]:
import cv2
import mediapipe as mp
import numpy as np

mp_draw = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def knee_angle(p1, p2, p3):
    if p1.visibility < 0.8 or p2.visibility < 0.8 or p3.visibility < 0.8:
        return -1
    a = np.array([p1.x - p2.x, p1.y - p2.y, p1.z - p2.z])
    b = np.array([p3.x - p2.x, p3.y - p2.y, p3.z - p2.z])
    ang = np.degrees(np.arccos(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))))
    return 360 - ang if ang > 180 else ang

def leg_pos(angle):
    if angle < 0: return 0
    if angle < 105: return 1
    if angle < 150: return 2
    return 3

# Map angle to depth percent
def squat_depth_percent(angle, max_angle=170, min_angle=90):
    if angle < 0:
        return 0
    # Clamp angle range
    angle = max(min_angle, min(max_angle, angle))
    # Invert so smaller angle -> bigger percent
    return int(((max_angle - angle) / (max_angle - min_angle)) * 100)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No camera!")
    exit()

count = 0
last_state = 9

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ok, frame = cap.read()
        if not ok:
            print("Camera error")
            break

        frame = cv2.resize(frame, (960, 540))
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb.flags.writeable = False
        results = pose.process(rgb)
        rgb.flags.writeable = True

        depth_percent = 0  # Default depth

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            r_ang = knee_angle(lm[24], lm[26], lm[28])
            l_ang = knee_angle(lm[23], lm[25], lm[27])

            rs = leg_pos(r_ang)
            ls = leg_pos(l_ang)
            state = rs * ls

            # Use average of both legs for depth
            if r_ang > 0 and l_ang > 0:
                avg_angle = (r_ang + l_ang) / 2
                depth_percent = squat_depth_percent(avg_angle)

            if state in (1, 9) and state != last_state:
                last_state = state
                if state == 1:
                    count += 1
                    print(f"Squat! Total: {count}")
            elif state == 0:
                if rs == 0: print("Right leg lost")
                if ls == 0: print("Left leg lost")

        else:
            print("Step into view")

        # Draw squat counter
        cv2.putText(frame, f"Reps: {count}", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw depth progress bar
        bar_x = frame.shape[1] - 50
        bar_height = int((depth_percent / 100) * frame.shape[0])
        cv2.rectangle(frame, (bar_x, frame.shape[0] - bar_height),
                      (bar_x + 30, frame.shape[0]), (0, 255, 0), -1)
        cv2.rectangle(frame, (bar_x, 0), (bar_x + 30, frame.shape[0]), (255, 255, 255), 2)
        cv2.putText(frame, f"{depth_percent}%", (bar_x - 10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        cv2.imshow("Squat Counter", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost

In [4]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe pose model
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Helper function to calculate angle between 3 points
def calculate_angle(a, b, c):
    """
    Calculates the angle at point 'b' given three coordinates:
    a (hip), b (knee), c (ankle) or similar joints.
    """
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point

    # Vector calculations
    ba = a - b
    bc = c - b

    # Calculate angle in radians and convert to degrees
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Open webcam
cap = cv2.VideoCapture(0)

# Variables for squat counting
counter = 0
stage = None  # "up" or "down"

# Depth bar configuration
bar_max_width = 300
ideal_knee_angle = 90  # Ideal squat depth

# Start pose detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the BGR frame to RGB for MediaPipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Perform pose detection
        results = pose.process(image)

        # Convert image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for left leg joints
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate knee angle
            knee_angle = calculate_angle(hip, knee, ankle)

            # Squat counter logic
            if knee_angle > 160:
                stage = "up"
            if knee_angle < 100 and stage == 'up':
                stage = "down"
                counter += 1

            # ---- Form Depth Feedback ----
            # Calculate how close the knee angle is to ideal depth
            form_score = max(0, min(100, 100 - abs(knee_angle - ideal_knee_angle)))
            bar_width = int((form_score / 100) * bar_max_width)

            # Choose bar color based on form score
            if form_score > 80:
                bar_color = (0, 255, 0)  # Green for good form
                feedback = "Perfect Depth ✅"
            elif form_score > 50:
                bar_color = (0, 165, 255)  # Orange for medium
                feedback = "Go Lower ⬇️"
            else:
                bar_color = (0, 0, 255)  # Red for bad form
                feedback = "Too Shallow ⚠️"

            # Draw form bar
            cv2.rectangle(image, (50, 50), (50 + bar_max_width, 80), (200, 200, 200), 2)
            cv2.rectangle(image, (50, 50), (50 + bar_width, 80), bar_color, -1)

            # Display feedback text
            cv2.putText(image, feedback, (50, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, bar_color, 2)

        except:
            pass

        # Display squat count
        cv2.putText(image, f"Squats: {counter}", (10, 450),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)

        # Draw pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Show image
        cv2.imshow('Squat Counter with Depth Feedback', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
